In [1]:
#!pip install ultralytics

In [2]:
import ultralytics
ultralytics.__version__

'8.3.97'

In [3]:
import cv2
import numpy as np
import pandas as pd
import time
import paho.mqtt.client as mqtt
from ultralytics import YOLO
from tracker import*

model=YOLO('yolo11l.pt')

In [4]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [5]:
tracker=Tracker()
count=0

In [6]:
cap1=cv2.VideoCapture(1)
cap2=cv2.VideoCapture('testvid2.mp4')

In [7]:
down={}
up={}

counter_down=[]
counter_up=[]

In [8]:
# MQTT Configuration
mqtt_broker = "broker.hivemq.com"  # Replace with your MQTT broker address
mqtt_port = 1883
mqtt_topic = "vehicle_counter/counter11"  # Topic for counter11 data
mqtt_client_id = "vehicle_counter_client"

# Initialize MQTT client
client = mqtt.Client(client_id=mqtt_client_id)

# Callback functions for MQTT client
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected to MQTT Broker!")
    else:
        print(f"Failed to connect, return code {rc}")

def on_publish(client, userdata, mid):
    print(f"Message {mid} published")

# Set callbacks
client.on_connect = on_connect
client.on_publish = on_publish

# Connect to MQTT broker
try:
    client.connect(mqtt_broker, mqtt_port, 60)
    client.loop_start()  # Start the loop in a background thread
except Exception as e:
    print(f"Failed to connect to MQTT broker: {e}")

# Track time for publishing delay
last_publish_time = time.time()
publish_interval = 3.0  # 3 seconds delay between publications

counter11=0
counter21=0
counter31=0
counter12=0
counter22=0
counter32=0
area_1=[(259,138),(389,137),(400,242),(342,466),(11,433)]
area_2=[(420,140),(477,126),(549,250),(587,451),(417,456),(433,252)]
area_3=[(895,238),(978,294),(869,482),(615,486),(700,337)]

# Assuming cap1, cap2, model, class_list, and tracker are defined elsewhere in your code
cap1 = cv2.VideoCapture(1)  # Replace with your actual video source
#cap2 = cv2.VideoCapture(1)  # Replace with your actual video source
count = 0

# Assuming model, class_list, and tracker are initialized elsewhere
# Replace these with your actual initialization code
# model = ...
# class_list = ...
# tracker = ...

# Initialize serial connection if needed
# ser = serial.Serial(...)

while True:    
    
    ret1,frame1 = cap1.read()
    ret2,frame2 = cap2.read()
    if not ret1 or not ret2:
        break
    count += 1
    frame1=cv2.resize(frame1,(1020,500))
    frame2=cv2.resize(frame2,(1020,500))

    results1=model.predict(frame1)
    results2=model.predict(frame2)

    a1=results1[0].boxes.data
    a1 = a1.detach().cpu().numpy()  # added this line
    px1=pd.DataFrame(a1).astype("float")

    a2=results2[0].boxes.data
    a2 = a2.detach().cpu().numpy()  # added this line
    px2=pd.DataFrame(a2).astype("float")

    list1=[]
    list2=[]
             
    for index,row in px1.iterrows():
        x11=int(row[0])
        y11=int(row[1])
        x21=int(row[2])
        y21=int(row[3])
        d1=int(row[5])
        c1=class_list[d1]
        if 'mouse' in c1:
            list1.append([x11,y11,x21,y21])
            
    for index,row in px2.iterrows():
        x12=int(row[0])
        y12=int(row[1])
        x22=int(row[2])
        y22=int(row[3])
        d2=int(row[5])
        c2=class_list[d2]
        if 'car' or 'truck' or 'bus' in c2:
            list2.append([x12,y12,x22,y22])

    cv2.polylines(frame2, [np.array(area_1, np.int32)], True, (15,220,10) ,2)
    cv2.polylines(frame2, [np.array(area_2, np.int32)], True, (15,220,10) ,2)
    cv2.polylines(frame2, [np.array(area_3, np.int32)], True, (15,220,10) ,2)

    bbox_id1=tracker.update(list1)
    for bbox in bbox_id1:
        x31,y31,x41,y41,id=bbox
        cx1=int(x31+x41)//2
        cy1=int(y31+y41)//2

        cv2.rectangle(frame1,(x31,y31), (x41,y41),(0,0,255),2)
        counter11+=1

    # Check if enough time has passed since the last publication
    current_time = time.time()
    if current_time - last_publish_time >= publish_interval:
        # Publish counter11 data to MQTT
        try:
            # Convert counter to string for MQTT payload
            payload = str(counter11)
            # Publish to the MQTT topic
            result = client.publish(mqtt_topic, payload)
            # Check if the message was published
            status = result[0]
            if status == 0:
                print(f"Sent {payload} to topic {mqtt_topic}")
            else:
                print(f"Failed to send message to topic {mqtt_topic}")
            # Update the last publish time
            last_publish_time = current_time
        except Exception as e:
            print(f"MQTT publish error: {e}")

    cv2.putText(frame1,('a--')+str(counter11),(200,110),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)
    counter11=0
    
    # Note: We're not resetting counter11 here, but accumulating it between publications
    # Remove this line: counter11=0
    
    bbox_id2=tracker.update(list2)
    for bbox in bbox_id2:
        x32,y32,x42,y42,id=bbox
        cx2=int(x32+x42)//2
        cy2=int(y32+y42)//2

        result12=cv2.pointPolygonTest(np.array(area_1, np.int32), (int(cx2), int(cy2)), False)
        result22=cv2.pointPolygonTest(np.array(area_2, np.int32), (int(cx2), int(cy2)), False)
        result32=cv2.pointPolygonTest(np.array(area_3, np.int32), (int(cx2), int(cy2)), False)

        if result12>=0:
            cv2.rectangle(frame2,(x32,y32), (x42,y42),(0,0,255),2)
            cv2.circle(frame2,(cx2,cy2),4,(0,0,255),-1)
            counter12+=1

        if result22>=0:
            cv2.rectangle(frame2,(x32,y32), (x42,y42),(0,0,255),2)
            cv2.circle(frame2,(cx2,cy2),4,(0,0,255),-1)
            counter22+=1

        if result32>=0:
            cv2.rectangle(frame2,(x32,y32), (x42,y42),(0,0,255),2)
            cv2.circle(frame2,(cx2,cy2),4,(0,0,255),-1)
            counter32+=1

    cv2.putText(frame2,('a--')+str(counter12),(200,110),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1, cv2.LINE_AA)
    cv2.putText(frame2,('b--')+str(counter22),(475,50),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA)
    cv2.putText(frame2,('c--')+str(counter32),(800,216),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
    counter12=0
    counter22=0
    counter32=0
    
    cv2.imshow("frames1", frame1)
    cv2.imshow("frames2", frame2)

    if cv2.waitKey(1)&0xFF==27:
        break

# Clean up
cap1.release()
cap2.release()  # Added this line
if 'ser' in locals():  # Check if ser exists
    ser.close()
client.loop_stop()  # Stop the MQTT client loop
client.disconnect()  # Disconnect from MQTT broker
cv2.destroyAllWindows()

C:\Users\Kaushal\AppData\Local\Temp\ipykernel_9380\1242342550.py:8: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client(client_id=mqtt_client_id)


Connected to MQTT Broker!

0: 320x640 1 person, 1 handbag, 1 bed, 3 mouses, 440.5ms
Speed: 4.7ms preprocess, 440.5ms inference, 2.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 10 cars, 1 truck, 394.2ms
Speed: 3.0ms preprocess, 394.2ms inference, 1.9ms postprocess per image at shape (1, 3, 320, 640)
Sent 3 to topic vehicle_counter/counter11
Message 1 published

0: 320x640 1 person, 1 handbag, 1 bed, 3 mouses, 367.1ms
Speed: 2.3ms preprocess, 367.1ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 11 cars, 1 truck, 351.6ms
Speed: 2.9ms preprocess, 351.6ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 handbag, 1 bed, 1 mouse, 311.4ms
Speed: 2.2ms preprocess, 311.4ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 11 cars, 1 truck, 299.1ms
Speed: 2.3ms preprocess, 299.1ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 handbag, 1 bed, 2 mouses, 301.5ms

KeyboardInterrupt: 

Connected to MQTT Broker!
